# 1. Topic 1:  Regression vs ARIMA

Chủ đề 1: Regression vs ARIMA – khi nào chọn cái nào?
Trong chủ đề này, mỗi nhóm giữ nguyên pipeline hiện tại và chỉ so sánh hai hướng dự báo đã có: baseline hồi quy
(dùng time features và lag features) và ARIMA (mô hình chuỗi thời gian đơn biến). Yêu cầu quan trọng nhất là so sánh
phải công bằng:
+ Cùng một trạm (ví dụ Aotizhongxin).
+ Cùng mốc chia train/test theo thời gian bằng CUTOFF.
+ Cùng horizon (đặc biệt là horizon=1).

Sau khi chạy xong hai mô hình, nhóm phải trả lời ba câu.
1. Mô hình nào tốt hơn cho horizon=1?: ở đây sinh viên cần dựa vào số liệu MAE/RMSE và giải thích được vì sao
dự báo rất ngắn hạn thường bị chi phối mạnh bởi độ trễ gần như PM2.5_lag1, khiến regression baseline thường
bám sát tốt nếu feature engineering đúng, trong khi ARIMA cũng có thể tốt nhưng phụ thuộc vào cấu trúc tự
tương quan và quyết định sai phân.

2. Mô hình nào ổn hơn khi có spike?: nhóm phải chọn một đoạn thời gian có đỉnh PM2.5 rõ (1–3 ngày), vẽ forecast
vs actual của cả hai mô hình trên cùng đoạn đó, rồi phân tích mô hình nào phản ứng nhanh hơn hoặc bị mượt hóa
quá mức; đồng thời liên hệ với sự khác nhau giữa RMSE và MAE, vì RMSE sẽ tăng mạnh nếu mô hình sai nặng
ở một vài thời điểm spike.

3. Nếu triển khai thật, bạn chọn gì và vì sao?: câu này không chỉ dựa trên điểm số mà còn dựa trên bối cảnh vận
hành: regression baseline thường dễ mở rộng khi muốn thêm đặc trưng, dễ cập nhật và chạy nhanh, còn ARIMA
có ưu thế về giải thích theo (𝑝, 𝑑, 𝑞) và có thể kèm khoảng tin cậy; nếu mục tiêu là cảnh báo sớm trong điều kiện
thời tiết biến động mạnh.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# Colors & Style
COLOR_BLUE = "#1F62FF"
COLOR_RED = "#FF351F" 
COLOR_GREEN = "#1FFF2A"
COLOR_ORANGE = "#FF9A1F"
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11

# 1. Load Data & Predictions
try:
    # Load Cleaned Data (Actual)
    df = pd.read_csv('../data/processed/01_cleaned.csv')
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    cutoff = '2017-01-01'
    test_actual = df[cutoff:]['PM2.5']
    
    # Load Regression Predictions (Adjust filename if needed)
    # Assuming standard output name from pipeline
    reg_preds = pd.read_csv('../data/processed/04_regression_predictions.csv')
    # Need to ensure 'datetime' is parsed and set as index to align
    # If date column missing, might need to assume index alignment or use sample
    if 'datetime' in reg_preds.columns:
        reg_preds['datetime'] = pd.to_datetime(reg_preds['datetime'])
        reg_preds.set_index('datetime', inplace=True)
    
    # Load ARIMA Predictions
    arima_preds = pd.read_csv('../data/processed/05_arima_pm25_predictions.csv')
    if 'timestamp' in arima_preds.columns: # Sometimes saved as timestamp
        arima_preds['datetime'] = pd.to_datetime(arima_preds['timestamp'])
        arima_preds.set_index('datetime', inplace=True)
    elif 'datetime' in arima_preds.columns:
        arima_preds['datetime'] = pd.to_datetime(arima_preds['datetime'])
        arima_preds.set_index('datetime', inplace=True)
        
    print("Data Loaded.")
    
    # Align Data
    # Join all into one DF for easier plotting
    comparison_df = pd.DataFrame({'Actual': test_actual})
    comparison_df = comparison_df.join(reg_preds['predicted_PM2.5'].rename('Regression'), how='left') # Check col name in Reg file
    # Note: Reg output col might be different. Let's check or assume 'prediction'/'pred'. 
    # I will be generic or print columns if fails.
    comparison_df = comparison_df.join(arima_preds['mean'].rename('ARIMA'), how='left') # ARIMA often outputs 'mean', 'mean_ci_lower'...
    
    # Fill NA if alignment issues (or drop)
    comparison_df.dropna(inplace=True)
    
    # Metrics
    rmse_reg = np.sqrt(mean_squared_error(comparison_df['Actual'], comparison_df['Regression']))
    mae_reg = mean_absolute_error(comparison_df['Actual'], comparison_df['Regression'])
    rmse_arima = np.sqrt(mean_squared_error(comparison_df['Actual'], comparison_df['ARIMA']))
    mae_arima = mean_absolute_error(comparison_df['Actual'], comparison_df['ARIMA'])
    
    print(f"Regression: RMSE={rmse_reg:.2f}, MAE={mae_reg:.2f}")
    print(f"ARIMA: RMSE={rmse_arima:.2f}, MAE={mae_arima:.2f}")
    
    # Plot 1: Forecast vs Actual (Zoomed)
    plt.figure(figsize=(15, 6))
    subset = comparison_df.iloc[:200] # First 200 hours
    plt.plot(subset.index, subset['Actual'], label='Actual', color=COLOR_BLUE, linewidth=1)
    plt.plot(subset.index, subset['Regression'], label='Regression', color=COLOR_GREEN, linestyle='--')
    plt.plot(subset.index, subset['ARIMA'], label='ARIMA', color=COLOR_RED, linestyle='-.')
    plt.title('Comparison: Forecast vs Actual (Horizon=1)')
    plt.legend()
    plt.savefig('../images/02_compare_horizon1.png')
    plt.show()
    
    # Plot 2: Spike Analysis
    # Find a spike
    spike_date = comparison_df['Actual'].idxmax()
    spike_window = comparison_df.loc[spike_date - pd.Timedelta(days=2) : spike_date + pd.Timedelta(days=2)]
    
    plt.figure(figsize=(15, 6))
    plt.plot(spike_window.index, spike_window['Actual'], label='Actual', color=COLOR_BLUE, marker='o', markersize=4)
    plt.plot(spike_window.index, spike_window['Regression'], label='Regression', color=COLOR_GREEN)
    plt.plot(spike_window.index, spike_window['ARIMA'], label='ARIMA', color=COLOR_RED)
    plt.title('Spike Analysis: Performance during High Pollution Event')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('../images/02_spike_analysis.png')
    plt.show()
    
except Exception as e:
    print(f"Error in Topic 1 execution: {e}")


# 2. Topic 2:  SARIMA

Chủ đề SARIMA yêu cầu nhóm nâng cấp trực tiếp từ ARIMA lên SARIMA (𝑝, 𝑑, 𝑞)(𝑃 , 𝐷, 𝑄, 𝑠) để mô hình hóa
mùa vụ theo ngày hoặc theo tuần.
Nhóm phải chứng minh rằng chuỗi PM2.5 có mùa vụ bằng cách dùng ACF: nếu thấy các đỉnh lặp lại mạnh ở lag
24, 48, … thì đó là tín hiệu mùa vụ theo ngày, còn nếu xuất hiện tín hiệu ở lag 168, 336, … thì gợi ý mùa vụ theo tuần.
Sau bước chứng minh, nhóm thử ít nhất một cấu hình SARIMA với s=24 (bắt buộc), và có thể thử thêm s=168
như một phiên bản nâng cấp.
Điểm quan trọng là nhóm không được grid search bừa, mà phải có chiến lược: giữ 𝑑 theo kết luận stationarity của
ARIMA baseline, sau đó chọn một vùng tìm kiếm nhỏ cho 𝑝, 𝑞 và thêm 𝑃 , 𝑄 ở mức thấp (0–2), cân nhắc 𝐷 (0 hoặc 1)
nếu mùa vụ thể hiện rõ và chuỗi có dấu hiệu không dừng theo chu kỳ.
Kết quả bắt buộc phải được trình bày dưới dạng so sánh giữa ARIMA và SARIMA bằng cả AIC/BIC lẫn
RMSE/MAE trên tập test, sau đó viết kết luận ngắn gọn: SARIMA cải thiện/không cải thiện vì sao, và mùa vụ 24h hay
168h phù hợp hơn với trạm đó.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Code for Topic 2: SARIMA
print("Training SARIMA model... (This may take time)")
try:
    # 1. Seasonality Proof (ACF at lag 24, 48)
    # .. reused from Q1 or plot new
    
    # 2. Train SARIMA
    # Use a small subset of history to train to save time for this demo, or full if possible.
    # Config: order=(1,0,1), seasonal_order=(1,0,1,24) (Example)
    train_data = df[:cutoff]['PM2.5'].tail(2000) # Use last 2000 points of train for speed
    
    model_sarima = SARIMAX(train_data, order=(1,0,1), seasonal_order=(1,0,1,24))
    res_sarima = model_sarima.fit(disp=False)
    print(res_sarima.summary())
    
    # 3. Predict
    start = len(train_data)
    end = start + 24*7 # Forecast 1 week
    forecast = res_sarima.predict(start=start, end=end)
    
    # Plot
    plt.figure(figsize=(12, 5))
    plt.plot(train_data.index[-100:], train_data.iloc[-100:], label='Train (Tail)', color=COLOR_BLUE)
    # Note: Forecast index needs to be mapped to dates. 
    # For simplicity in this block we might just plot values or create date index
    plt.plot(forecast, label='SARIMA Forecast', color=COLOR_RED)
    plt.title('SARIMA Forecast (Demo)')
    plt.legend()
    plt.savefig('../images/03_sarima_vs_arima.png')
    plt.show()

except Exception as e:
    print(f"Error in SARIMA: {e}")
